In [26]:
import json
import pandas as pd

from nltk.corpus import wordnet

from textblob import TextBlob
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix 
from sklearn.feature_extraction.text import CountVectorizer
from textblob import Word
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
%matplotlib inline

from gensim import corpora, models, matutils
from os import path

/home/ec2-user/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [4]:
f = open('161103_times.json','r')

In [5]:
filestring = f.read()

In [6]:
split = filestring.replace(']\n','').replace('[\n','').split('\n')


In [7]:
final_dict = {}

In [8]:
for i in split:
    try:
        temp = json.loads(i[:-1])
    except:
        continue   
    val = temp.values()[0].encode('utf-8').replace("\xe2\x80\x99","'").replace("\xe2\x80\x9c",'"').replace("\xe2\x80\x9d",'"').replace('\xe2\x80\x94',"'")
    key = temp.keys()[0]
    if val != '':
        final_dict[key] = val
    else:
        continue
    
    
    

In [9]:
dump_dict = json.dumps(final_dict)
df = pd.read_json(dump_dict, typ = 'series')

In [10]:
df = pd.DataFrame(df).reset_index()
df.columns  = [['url','text']]

In [11]:
df['date'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[:3])
df['section'] = df['url'].apply(lambda x: x.split('.com/')[1].split('/')[3])
df['length'] = df['text'].apply(lambda x: len(x))


In [14]:
df['nltk'] = df['text'].apply(lambda x: word_tokenize(x) )

In [16]:
df['tagged'] = df['nltk'].apply(lambda x: nltk.pos_tag(x))

In [142]:
lemmatizer = WordNetLemmatizer()
def lemmatize_me(x):
    return_list = []
    for i in x:
#         print i
        if 'V' in i[1]:
            value = 'v'
        else:
            value = 'n'
        return_list.append(lemmatizer.lemmatize(i[0], pos = value))
    return return_list
        


In [143]:
df['lemmas'] = df['tagged'].apply(lambda x: lemmatize_me(x))

In [19]:
df['senlem'] = df['lemmas'].apply(lambda x: ' '.join(x))

In [145]:
df['nltk'][0]

[u'The',
 u'renowned',
 u'chef',
 u'Ren\xe9',
 u'Redzepi',
 u'of',
 u'the',
 u'two-Michelin-starred',
 u'Noma',
 u'in',
 u'Copenhagen',
 u'was',
 u"n't",
 u'exaggerating',
 u'when',
 u'he',
 u'said',
 u',',
 u'``',
 u'People',
 u'will',
 u'travel',
 u'anywhere',
 u'for',
 u'good',
 u'food',
 u"'",
 u'it',
 u"'s",
 u'crazy',
 u'.',
 u"''",
 u'Indeed',
 u',',
 u'food',
 u'lovers',
 u'are',
 u'now',
 u'trooping',
 u'around',
 u'the',
 u'world',
 u'to',
 u'both',
 u'accessible',
 u'and',
 u'remote',
 u'destinations',
 u'.',
 u'Perhaps',
 u'Mr.',
 u'Redzepi',
 u'should',
 u'have',
 u'added',
 u'that',
 u'overindulging',
 u'is',
 u'almost',
 u'certainly',
 u'the',
 u'case',
 u'with',
 u"'",
 u'and',
 u'maybe',
 u'the',
 u'point',
 u'of',
 u"'",
 u'any',
 u'culinary-inspired',
 u'trip',
 u'.',
 u'It',
 u"'s",
 u'hard',
 u',',
 u'for',
 u'example',
 u',',
 u'to',
 u'hang',
 u'on',
 u'to',
 u'discipline',
 u'at',
 u'the',
 u'in',
 u'Italy',
 u"'s",
 u'Piedmont',
 u'region',
 u'.',
 u'Coming',
 

In [111]:
cv = CountVectorizer(analyzer = 'word',stop_words='english',min_df=0.05, max_df = 0.8).fit(df['senlem'])

In [112]:
# cv_alt = CountVectorizer(analyzer=lemmatize_me).fit(df['tagged'])

In [113]:
sm_test = cv.transform(df['senlem'])
sm_transpose = cv.transform(df['senlem']).transpose()

In [114]:
print "shape", sm_test.shape
print "nonzeros", sm_test.nnz
print "sparsity", (100.0 * sm_test.nnz / (sm_test.shape[0] * sm_test.shape[1]))
print '\n'
print "trans shape", sm_transpose.shape
print "trans nonzero", sm_transpose.nnz
print "sparsity", (100.0 * sm_transpose.nnz/ (sm_transpose.shape[0] * sm_transpose.shape[1]))

shape (561, 1226)
nonzeros 82065
sparsity 11.931763659


trans shape (1226, 561)
trans nonzero 82065
sparsity 11.931763659


In [115]:
corpus = matutils.Sparse2Corpus(sm_transpose)
id2word = dict((v, k) for k, v in cv.vocabulary_.iteritems())

In [138]:
lda = models.LdaModel(corpus, id2word=id2word, num_topics=25, passes=10, alpha='auto', eta='auto')


In [137]:
sorted(lda.show_topic(9, topn=20), key = lambda x: x[1], reverse=True)

[(u'dr', 0.014488328842092155),
 (u'people', 0.01228331696287714),
 (u'year', 0.010590434977096184),
 (u'state', 0.010315968200221586),
 (u'work', 0.0085261074798596587),
 (u'mr', 0.008337592871637968),
 (u'time', 0.0077330011111772399),
 (u'make', 0.006620402160900634),
 (u'know', 0.0065448531378805011),
 (u'like', 0.006266977191972313),
 (u'use', 0.0058659150723740332),
 (u'new', 0.0057331826001435528),
 (u'help', 0.0055307023548295167),
 (u'ms', 0.0054732013467348817),
 (u'report', 0.0049537522485215355),
 (u'government', 0.004898761480363732),
 (u'study', 0.0048361057325232125),
 (u'week', 0.0048323812517277248),
 (u'day', 0.0047853881480714984),
 (u'treatment', 0.0047133498492918516)]

In [30]:
lda_corpus = lda[corpus]

In [31]:
lda_docs = [doc for doc in lda_corpus]

In [32]:
doc_topics = [[df['url'][i], lda_docs[i]] for i in range(len(lda_docs))]

In [33]:
for i in doc_topics:
    print i[0], sorted(lda.show_topic((max(i[1], key=lambda x: x[1])[0]), topn=10), key = lambda x: x[1], reverse=True)

http://www.nytimes.com/2016/01/10/travel/places-to-visit-food.html?ref=dining [(u'food', 0.011623857447438957), (u'make', 0.010817042240062308), (u'say', 0.010769304093918471), (u'chef', 0.0073237559894846647), (u'mr', 0.0072826347156430288), (u'like', 0.0063908193838190094), (u'cook', 0.0062488956396173165), (u'dish', 0.0060667149876988662), (u'restaurant', 0.0059652339987380804), (u'just', 0.0055638462634365884)]
http://www.nytimes.com/2016/07/25/world/asia/india-transgender.html?ref=world [(u'mr', 0.022081605572683113), (u'say', 0.019846934291259114), (u'state', 0.0080854642640126213), (u'year', 0.0073129348308877146), (u'trump', 0.0061725702724323246), (u'clinton', 0.0056434269404687168), (u'time', 0.0050468228808338142), (u'make', 0.0049601850109408001), (u'american', 0.0048485072216979458), (u'new', 0.0042254947860450465)]
http://www.nytimes.com/2016/07/31/health/harnessing-the-immune-system-to-fight-cancer.html [(u'dr', 0.017550422732894238), (u'say', 0.016522404332026914), (u'c

In [365]:
tt = 

<561x1227 sparse matrix of type '<type 'numpy.float64'>'
	with 82526 stored elements in Compressed Sparse Row format>

In [80]:
corpus = matutils.Sparse2Corpus(tt.transpose())
# id2word = dict((v, k) for k, v in cv.vocabulary_.iteritems())
lda = models.LdaModel(corpus, num_topics=35, passes=10, alpha='auto', eta='auto')
lda_corpus = lda[corpus]
doc_topics = [[df['url'][i], lda_docs[i]] for i in range(len(lda_docs))]
for i in doc_topics:
    print i[0], sorted(lda.show_topic((max(i[1], key=lambda x: x[1])[0]), topn=10), key = lambda x: x[1], reverse=True)

http://www.nytimes.com/2016/01/10/travel/places-to-visit-food.html?ref=dining [('579', 0.0014989105992328159), ('315', 0.0014144993977060136), ('200', 0.0012631165001477265), ('976', 0.0012375102619223205), ('103', 0.0012156032553791312), ('1185', 0.0011872418361344199), ('959', 0.0010978079881777403), ('776', 0.0010736140154876451), ('735', 0.0010646037350223736), ('247', 0.0010540072545894238)]
http://www.nytimes.com/2016/07/25/world/asia/india-transgender.html?ref=world [('432', 0.0026090049868277725), ('605', 0.001700000762936663), ('984', 0.001485184866518618), ('902', 0.0014776199891674161), ('64', 0.0014199371055511542), ('84', 0.0013934113642727806), ('266', 0.001391944781025825), ('1010', 0.0012771479338997852), ('1029', 0.001228902058329375), ('628', 0.0012169268672037759)]
http://www.nytimes.com/2016/07/31/health/harnessing-the-immune-system-to-fight-cancer.html [('755', 0.0013511008405719042), ('452', 0.0013255069130802899), ('748', 0.0013162575762562154), ('81', 0.00109350

In [41]:
from sklearn.cluster import KMeans


In [42]:
kmeans = KMeans(n_clusters=50)

In [82]:
km = kmeans.fit_transform(tt)

In [83]:
mu_articles =  km.cluster_centers_


In [84]:
mu_articles.shape

(50, 1227)

In [369]:
kmeans.fit_transform(tt).shape

(561, 50)

In [85]:
lsa = TruncatedSVD(n_components=250)
nyt_lsa = lsa.fit_transform(tt)
nyt_lsa = Normalizer(copy=False).fit_transform(nyt_lsa)



In [139]:
lsa.explained_variance_ratio_.shape

(250,)

In [89]:
pd.DataFrame(lsa.components_.round(5), columns = cv.get_feature_names())

,000,10,100,11,12,13,14,15,16,17,...,worst,worth,write,writer,wrong,year,yes,york,young,younger
0,0.05310,0.03795,0.01835,0.01833,0.01965,0.01182,0.01076,0.02380,0.01163,0.01040,...,0.00932,0.01722,0.04484,0.01635,0.01427,0.13212,0.01381,0.06126,0.03257,0.01162
1,-0.00145,-0.02535,-0.00133,-0.00397,-0.01443,-0.00291,-0.00120,-0.01668,-0.00628,-0.00841,...,0.00454,-0.01725,-0.00190,-0.01277,0.00793,-0.00724,-0.00317,-0.01331,-0.00770,0.00137
2,0.02281,0.02307,0.00351,0.00507,0.00364,0.00190,0.00688,0.00596,0.00175,0.01107,...,0.00603,-0.00684,0.00561,-0.00237,-0.00862,0.05260,0.00700,-0.03454,0.02117,0.00126
3,-0.07140,0.02488,-0.00676,0.00761,-0.00404,0.00113,0.00626,-0.00027,0.00171,0.01076,...,-0.00428,0.00300,-0.00461,0.01954,0.02276,-0.03239,0.01207,-0.02379,0.00474,0.00030
4,-0.01570,-0.00213,-0.01272,-0.00515,-0.00731,-0.00283,-0.00386,-0.00090,0.00012,0.00427,...,-0.00734,-0.00452,-0.01249,-0.00367,-0.01238,-0.00377,-0.00755,0.00849,0.01069,-0.00427
5,-0.01944,-0.00659,0.00493,-0.00957,-0.01169,0.00050,0.00743,-0.00796,-0.00423,0.00291,...,-0.00036,0.01006,-0.00904,-0.00400,0.00636,0.02346,-0.01218,-0.03906,-0.00267,-0.00070
6,0.07030,0.02841,0.01618,0.00604,0.01242,0.01036,0.01259,0.02221,0.00400,0.00767,...,0.00346,0.00199,-0.04106,-0.00668,-0.00318,0.07090,0.00603,0.06029,0.01575,0.00680
7,-0.06045,-0.01315,-0.01015,-0.01213,-0.01645,-0.00999,-0.00684,-0.00965,-0.00687,-0.01586,...,-0.00684,0.00462,0.04481,0.02836,0.01311,-0.00355,0.00485,-0.04532,0.03459,0.01290
8,0.05518,0.02403,0.02031,0.00314,0.01434,0.00896,0.00038,0.00281,0.00481,-0.00224,...,0.00328,0.01276,-0.02872,-0.03123,-0.01356,0.05586,-0.00126,-0.00461,-0.04381,-0.01195
9,0.00039,-0.00585,-0.01109,0.01588,0.00244,-0.00274,-0.01238,-0.01142,0.00266,-0.01717,...,0.01198,0.02034,0.03218,0.02750,0.00484,-0.01074,0.01199,-0.00657,0.00809,0.00351


In [280]:
tokens = tokenize(df['text'][0])

In [281]:
tokens

['The',
 'renowned',
 'chef',
 'Ren\xc3\xa9',
 'Redzepi',
 'of',
 'the',
 'two-Michelin-starred',
 'Noma',
 'in',
 'Copenhagen',
 'wasn',
 "'t",
 'exaggerating',
 'when',
 'he',
 'said',
 ',',
 '"',
 'People',
 'will',
 'travel',
 'anywhere',
 'for',
 'good',
 'food',
 "'",
 'it',
 "'s",
 'crazy',
 '.',
 '"',
 'Indeed',
 ',',
 'food',
 'lovers',
 'are',
 'now',
 'trooping',
 'around',
 'the',
 'world',
 'to',
 'both',
 'accessible',
 'and',
 'remote',
 'destinations',
 '.',
 'Perhaps',
 'Mr',
 'Redzepi',
 'should',
 'have',
 'added',
 'that',
 'overindulging',
 'is',
 'almost',
 'certainly',
 'the',
 'case',
 'with',
 "'",
 'and',
 'maybe',
 'the',
 'point',
 'of',
 "'",
 'any',
 'culinary-inspired',
 'trip',
 '.',
 'It',
 "'s",
 'hard',
 ',',
 'for',
 'example',
 ',',
 'to',
 'hang',
 'on',
 'to',
 'discipline',
 'at',
 'the',
 'in',
 'Italy',
 "'s",
 'Piedmont',
 'region',
 '.',
 'Coming',
 'off',
 'its',
 '85th',
 'anniversary',
 'last',
 'year',
 ',',
 'the',
 'more',
 'than',
 'mo

In [147]:
import sys, os
#!/usr/bin/python
# parent = os.path.dirname(os.path.realpath(__file__))
sys.path.append('~/mitie')

from mitie import *
from collections import defaultdict

ner = named_entity_extractor('../../../mitie/mitie-v0.2-python-2.7-windows-or-linux64/MITIE-models/english/ner_model.dat')

In [320]:
entities = ner.extract_entities()

TypeError: extract_entities() takes exactly 2 arguments (1 given)

In [283]:
entities

[(xrange(10, 11), 'LOCATION', 1.5750431281832638),
 (xrange(50, 52), 'PERSON', 0.29358908004603756),
 (xrange(89, 90), 'LOCATION', 1.9806472063153788),
 (xrange(91, 92), 'LOCATION', 0.9612849239684232),
 (xrange(195, 196), 'LOCATION', 1.7531823175672783),
 (xrange(201, 202), 'LOCATION', 1.5190761257406178),
 (xrange(209, 210), 'LOCATION', 1.3581381344870387),
 (xrange(298, 299), 'MISC', 0.996778869924363),
 (xrange(309, 311), 'LOCATION', 1.4581285876671708),
 (xrange(312, 313), 'LOCATION', 0.9695208907334698),
 (xrange(333, 335), 'LOCATION', 0.7135264236014621),
 (xrange(425, 427), 'LOCATION', 1.1283765103805443),
 (xrange(444, 445), 'LOCATION', 1.059132414917768),
 (xrange(461, 462), 'LOCATION', 0.547655303255666),
 (xrange(473, 475), 'LOCATION', 0.6249180511408932),
 (xrange(477, 479), 'LOCATION', 1.2447699841398736),
 (xrange(505, 507), 'LOCATION', 1.2329058295890734),
 (xrange(564, 565), 'MISC', 0.809707376043154),
 (xrange(566, 568), 'PERSON', 0.44325563264725765),
 (xrange(577, 5

In [330]:
def get_entities(x):
    x = tokenize(x)
    entities = ner.extract_entities(x)
    entities_list = []
    for e in entities:
        rangee = e[0]
#             print rangee
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(x[i] for i in rangee)
#             print "   Score: " + score_text + ": " + tag + ": " + entity_text
        entities_list.append(entity_text)
    return entities_list
#     final_list = []
#     for i in entities_list:
#         final_list.append(i)
#         final_list.extend(i.replace('.','').replace(',','').replace('"','').replace('(','').replace(')','').strip().split())
#     final = list(set(final_list))
#     return final

In [347]:
cv = CountVectorizer(analyzer = get_entities,stop_words='english', max_df = 0.8, min_df = 0.005, strip_accents='unicode', ngram_range=(1,3)).fit(df['text'])

In [339]:
sm = cv.transform(df['text'])

In [364]:
tfidf = tfidf_transformer.fit_transform(sm)


In [360]:
cv.vocabulary_

{'. States': 0,
 '650-seat Parliament': 1,
 'ABC': 2,
 'AOL': 3,
 'AT&T': 4,
 'Abdel Fattah': 5,
 'Abedin': 6,
 'Addison Russell': 7,
 'Advanced Studies Lucca': 8,
 'Affordable Care Act': 9,
 'Afghanistan': 10,
 'Afghans': 11,
 'Africa': 12,
 'African': 13,
 'African Export-Import Bank': 14,
 'African-American': 15,
 'African-Americans': 16,
 'Agrippa Muvhaku': 17,
 'Airbnb': 18,
 'Akamai': 19,
 'Alabama': 20,
 'Alaska': 21,
 'Alaskans': 22,
 'Alexa': 23,
 'Alexander Keyssar': 24,
 'All-Star': 25,
 'Allen': 26,
 'Allies': 27,
 'Almora': 28,
 'Alsace': 29,
 'Amazon': 30,
 'America': 31,
 'American': 32,
 'American Beverage Association': 33,
 'American Indian': 34,
 'American Revolution': 35,
 'American Society': 36,
 'Americas': 37,
 'Andrew M. Cuomo': 38,
 'Andrew Miller': 39,
 'Andy Ellis': 40,
 'Andy Warhol': 41,
 'Angela Merkel': 42,
 'Antarctica': 43,
 'Anthony Bourdain': 44,
 'Anthony D. Weiner': 45,
 'Anthony Rizzo': 46,
 'Anton Pi\xc3\xabch': 47,
 'Apple': 48,
 'Arab': 49,
 'Ara

In [359]:
len(cv.vocabulary_)

987